In [6]:
import os

from dotenv import load_dotenv

from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack import Pipeline
from haystack.utils import ComponentDevice
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever, InMemoryEmbeddingRetriever
from haystack.components.joiners import DocumentJoiner
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator


In [7]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

# Load documents

In [8]:
document_store = InMemoryDocumentStore()

docs = []
ARTICLES_DIR = os.path.join("data", "articles")
articles_titles = [name for name in os.listdir(ARTICLES_DIR)]

for title in articles_titles:
    path = os.path.join(ARTICLES_DIR, title)
    with open(path, "r") as file:
        content = file.read()
        docs.append(Document(content=content, meta={"title": title}))


# Process Documents

In [ ]:
EMBEDDING_MODEL = "avsolatorio/GIST-small-Embedding-v0"
# EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
RERANKER_MODEL = "BAAI/bge-reranker-base"

In [11]:
document_splitter = DocumentSplitter(split_by="word", split_length=512, split_overlap=32)
document_embedder = SentenceTransformersDocumentEmbedder(
    model=EMBEDDING_MODEL
)
document_writer = DocumentWriter(document_store)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("document_splitter", document_splitter)
indexing_pipeline.add_component("document_embedder", document_embedder)
indexing_pipeline.add_component("document_writer", document_writer)

indexing_pipeline.connect("document_splitter", "document_embedder")
indexing_pipeline.connect("document_embedder", "document_writer")

indexing_pipeline.run({"document_splitter": {"documents": docs}})

Batches: 100%|██████████| 59/59 [08:32<00:00,  8.68s/it]


{'document_writer': {'documents_written': 1887}}

# Create the Retriever

In [ ]:
text_embedder = SentenceTransformersTextEmbedder(
    model=EMBEDDING_MODEL
    #, device=ComponentDevice.from_str("cuda:0")
)
embedding_retriever = InMemoryEmbeddingRetriever(document_store)
bm25_retriever = InMemoryBM25Retriever(document_store)

In [ ]:
document_joiner = DocumentJoiner()
ranker = TransformersSimilarityRanker(model=RERANKER_MODEL)


In [15]:
hybrid_retrieval = Pipeline()
hybrid_retrieval.add_component("text_embedder", text_embedder)
hybrid_retrieval.add_component("embedding_retriever", embedding_retriever)
hybrid_retrieval.add_component("bm25_retriever", bm25_retriever)
hybrid_retrieval.add_component("document_joiner", document_joiner)
hybrid_retrieval.add_component("ranker", ranker)

hybrid_retrieval.connect("text_embedder", "embedding_retriever")
hybrid_retrieval.connect("bm25_retriever", "document_joiner")
hybrid_retrieval.connect("embedding_retriever", "document_joiner")
hybrid_retrieval.connect("document_joiner", "ranker")

# hybrid_retrieval.draw("hybrid-retrieval.png")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: InMemoryEmbeddingRetriever
  - bm25_retriever: InMemoryBM25Retriever
  - document_joiner: DocumentJoiner
  - ranker: TransformersSimilarityRanker
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (List[float])
  - embedding_retriever.documents -> document_joiner.documents (List[Document])
  - bm25_retriever.documents -> document_joiner.documents (List[Document])
  - document_joiner.documents -> ranker.documents (List[Document])

In [16]:
query = "Jordan Bardella est-il un bon candidat pour l'écologie ?"

result = hybrid_retrieval.run(
    {"text_embedder": {"text": query}, "bm25_retriever": {"query": query}, "ranker": {"query": query}}
)

Batches: 100%|██████████| 1/1 [00:04<00:00,  4.05s/it]


In [17]:
for doc in result.get("ranker").get("documents"):
    print("===================")
    print(doc.meta["title"])
    print(doc.content)


presidentielle-2022-course-a-la-nullite-des-candidates
grosses bêtises en 20 secondes
, en plaçant notamment la France comme meilleur élève écologique (c’est faux) et en confondant mix électrique et mix énergétique. Heureusement que c’est Jordan Bardella qui est aujourd’hui président du Rassemblement National le temps de la campagne, lui qui avait déclaré en juin dernier que “
Le C20 a été divisé par 1000 depuis les années 60
“. Une équipe prête à diriger la France, sans aucun doute.
Jean-Luc Mélenchon, l’écologie spectacle
S’il y a bien un candidat qui illustre à la perfection la politique des punchlines, des incantations et des petites phrases relayées sur les réseaux sociaux, c’est Jean-Luc Mélenchon.
Mais lorsqu’il s’agit de mettre en pratique les paroles, c’est un tout petit peu plus compliqué. Il y a un an, il proposait déjà de sortir du nucléaire en “
inventant une autre énergie bas-carbone
“. Le débat énergétique mérite mieux que d’attendre qu’Harry Potter nous sorte une énergi

# RAG

In [18]:
in_memory_retriever = InMemoryEmbeddingRetriever(document_store)

In [19]:
template = """
You are Bonbot, a virtual assistant that answers questions related to climate. The topics you cover are climate change, its impacts, and socio-economic news.

You must answer the question based on the excerpts of articles given in the context. You can only rely on the information contained in these excerpts.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

In [20]:
generator = OpenAIGenerator(model="gpt-4o", generation_kwargs={"temperature": 0, "max_tokens": 250})

In [21]:
retriever = InMemoryEmbeddingRetriever(document_store)
embedder = SentenceTransformersTextEmbedder(model="avsolatorio/GIST-small-Embedding-v0")

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
# TODO change retriever for hybrid
basic_rag_pipeline.add_component("text_embedder", embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [22]:
question = "Jordan Bardella est-il un bon candidat pour l'écologie ?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print("\n\n")
print(response["llm"]["replies"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.30it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)





Jordan Bardella n'est pas considéré comme un bon candidat pour l'écologie. Selon le contexte fourni, il est critiqué pour son manque de connaissance et d'activité au Parlement Européen, ainsi que pour ses positions qui ne soutiennent pas les objectifs climatiques. Son programme, s'il était appliqué, serait perçu comme un grand bond en arrière en matière de politique climatique. De plus, il est associé à des idées et des personnes qui ne sont pas alignées avec les efforts pour lutter contre le changement climatique.


In [17]:
document_store.save_to_disk("data/document_store")

In [20]:
new_store = InMemoryDocumentStore.load_from_disk("data/document_store")

"\n    Stores data in-memory. It's ephemeral and cannot be saved to disk.\n    "

# Giskard testset generation

In [23]:
from giskard.rag import KnowledgeBase, generate_testset, QATestset
import pandas as pd

knowledge_base_df = pd.DataFrame([doc[1].content for doc in document_store.storage.items()], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
testset = generate_testset(knowledge_base,
                           num_questions=120,
                           agent_description="A virtual assistant that answers questions related to climate change",
                           language="fr")

Generating questions: 100%|██████████| 120/120 [25:14<00:00, 12.62s/it]


In [26]:
# Save the testset
testset.save(os.path.join("data", "raget_testset_bilingue.jsonl"))

In [27]:
testset.to_pandas().to_csv(os.path.join("data", "raget_testset_bilingue_scored.csv"), sep="\t")

In [28]:
testset.to_pandas()

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
4e2979a2-b671-42ff-9e4d-d12b513fd251,Quelle est la position de Raphaël Glucksmann s...,Raphaël Glucksmann souhaite le déclin rapide d...,Document 1318: ont été abondamment questionnée...,[],"{'question_type': 'simple', 'seed_document_id'..."
a74fd226-11ef-4815-b1d5-1d6232ede2b3,Quels sont les principaux facteurs du réchauff...,L'intégralité du réchauffement global observé ...,Document 1585: évolué et ses possibles évoluti...,[],"{'question_type': 'simple', 'seed_document_id'..."
52d4623a-42ae-4a3b-8f9c-13f7fcc777f3,Qu'est-ce que la justice climatique et comment...,La justice climatique souligne l’importance du...,Document 1516: ne peut pas réussir une transit...,[],"{'question_type': 'simple', 'seed_document_id'..."
89bc0dc5-8f1e-4174-bc8f-1e955dc02ef1,Quels sont quelques exemples de changements de...,Parmi les exemples de changements de modes de ...,Document 706: celui qui veut que « changer no...,[],"{'question_type': 'simple', 'seed_document_id'..."
57bd3e7d-f37c-4457-802c-5d7c8a7120bf,Quelle est la conséquence la plus médiatisée d...,La conséquence la plus médiatisée de la défore...,Document 1532: L’Amazonie est peut-être le pre...,[],"{'question_type': 'simple', 'seed_document_id'..."
...,...,...,...,...,...
51945cf8-cd05-4a32-864d-c949072a2550,Quelle est-elle ?,Les inventaires nationaux sont établis par les...,"Document 72: – sociologie, économie, géographi...","[{'role': 'user', 'content': 'Je me demande à ...","{'question_type': 'conversational', 'seed_docu..."
ec98cd6d-93db-4585-a1f4-91b8e79a46fc,Pouvez-vous me dire ce que c'est?,Le JAITOUTCOMPRISME se définit comme toute per...,Document 638: d’ailleurs comme cela que devrai...,"[{'role': 'user', 'content': 'Je me réfère à u...","{'question_type': 'conversational', 'seed_docu..."
c5d327f8-edef-4882-89a2-f295b1350676,Qui en est le traducteur?,Le texte original a été traduit par M. Mobular.,Document 195: et ceux qui les ont amenés au po...,"[{'role': 'user', 'content': 'Je me demande au...","{'question_type': 'conversational', 'seed_docu..."


In [29]:
testset.to_pandas().to_excel(os.path.join("data", "raget_testset_bilingue_scored.xlsx"))

In [30]:
testset._questions

[QuestionSample(id='4e2979a2-b671-42ff-9e4d-d12b513fd251', question="Quelle est la position de Raphaël Glucksmann sur l'énergie nucléaire?", reference_answer='Raphaël Glucksmann souhaite le déclin rapide des énergies fossiles et le développement massif des énergies renouvelables, avec une part de nucléaire dans notre mix. Plus tôt dans son programme, il indique conserver une part de nucléaire comme une énergie de transition dans notre mix énergétique.', reference_context="Document 1318: ont été abondamment questionnées, comme lors du débat du 4 juin 2024 sur France 2 où cela a (encore) très largement dominé les débats.\nEn analysant les trois programmes, la liste de Manon Aubry (LFI) est la liste la plus en rupture avec le système européen actuel. Quand les deux listes EELV et Place Publique veulent “plus d’Europe” et tendre vers une “\nEurope fédérale\n“, LFI entend mener un combat en sortant des accords de libre-échange et en abrogeant les règles d’austérité “qui détruisent les servi